In [1]:
from google.cloud import bigquery
from time import time

## Parametros

In [2]:
proyecto = 'co-keralty-costomedico'
dataset = 'Diabetes_avicena_new'

# examenes

fecha_para_tomar_examenes = '2019-10-31'

examenes_glicemia = ['Glicemia Basal (mg/dl)','Glicemia (mg/dl)','Glucosa']
examenes_glucosa = ['Glicemia 2 horas Postprandial (mg/dl)','Glicemia Pos(mg/dl)']
examenes_hba1c = ['Hemoglobina Glicosilada (%)','Hemoglobina Glicosilada (HbA1c) (%)','Prueba rápida hemoglobina glicosilada']
examenes_hdl = ['Colesterol HDL','Colesterol HDL (mg/dl)','HDL (mg/dl)']
examenes_ldl = ['Colesterol LDL','Colesterol LDL (mg/dl)','LDL (mg/dl)']
examenes_trigliceridos = ['Triglicéridos','Triglicéridos (mg/dl)']
examenes_albuminuria = ['Microalbuminuria (mg/l)','Relaciones albumina/creatinina (RAC)','Relación Albuminuria/creatinuria (RAC)']

# Antecedentes

codigos_medicamentos_hipertension = ['C02AB0113C01','C02AB0113C02','C02AC0113C01','C02CA0113C01','C03EA0113C02',
                                     'C03EA0113C03','C07AA0513C01','C07AA0513C02','C07AB0213C02','C07AB0213C03',
                                     'C07AB0214C01','C07AB0214C02','C07AB0214C03','C07AB0214C04','C07AB0231C01',
                                     'C07AB0313C01','C07AB0313C02','C07AB0713C01','C07AB0713C07','C07CB0313C01',
                                     'C08CA0101C01','C08CA0101C02','C08CA0101C08','C08CA0113C01','C08CA0113C02',
                                     'C08CA0113C03','C08CA0113C04','C08CA0113C05','C08CA0113C07','C08CA0113C08',
                                     'C08CA0113C10','C08CA0501C01','C08CA0503C01','C08CA0514C02','C08DA0103C02',
                                     'C08DA0113C01','C08DA0113C03','C08DA0114C02','C08DA0131C01','C08GA0213C01',
                                     'C09AA0113C01','C09AA0113C02','C09AA0213C02','C09AA0213C03','C09BB0413C01',
                                     'C09BB0413C02','C09BB0413C03','C09BB0413C04','C09BB1003C01','C09BB1013C01',
                                     'C09BX0113C02','C09BX0113C03','C09CA0113C02','C09CA0113C03','C09CA0301C14',
                                     'C09CA0813C01','C09CA0813C02','C09CA0813C03','C09DA0113C02','C09DA0113C03',
                                     'C09DA0113C04','C09DA0613C01','C09DB0101C04','C09DB0113C01','C09DB0113C02',
                                     'C09DB0113C05','C09DB0413C02','C09DB0413C03','C09DB0413C05','C09DB0513C01',
                                     'C09DB0513C02','C09DB0513C03','C09DB0513C04','C09DB0601C01','C09DX0113C02',
                                     'C09DX0113C04','C09DX0113C05','MAGISTR6018']

codigos_cardiovascular = ['I630','I631','I632','I633','I634','I635','I638','I639','I653','I658','I659','I660','I661','I662','I664','I668',
                          'I669','I672','I693','I698','I500','I509','I518','Y840','I210','I211','I212','I213','I214','I219','I220','I221',
                          'I228','I229','I230','I231','I232','I233','I234','I235','I238','I240','I250','I251','I252','I702','I708','I709',
                          'I739','I236','I240','I513','I81X','J960','J961','J969','P285','E271','E273','E274','E283','E310','E894','I051',
                          'I052','I061','I062','I071','I110','I119','I120','I129','I130','I131','I132','I340','I351','I361','I371','I500',
                          'I501','I509','I872','J951','J952','J953','J960','J961','J969','K704','K720','K721','K729','N170','N171','N172',
                          'N178','N179','N180','N188','N189','N19X','N990','O084','O904','P285','P290','P960','Q222','Q231','Q233','Z353',
                          'Z597','I210','I211','I212','I213','I214','I219','I220','I221','I228','I229','I230','I231','I232','I252','I256',
                          'P294','I200','I201','I208','I209','I10X','I110','I119','I120','I129','I130','I131','I132','I139','I150','I151']

familiar_con_dm = ['Antecedente familiar de diabetes','!']

diabetes_diagnostico = ['Diabetes','!']

codigos_dm_gestacional = ['O240','O241','O242','O243','O244','O249','P700','P701','P702']

codigos_ovario_poliquistico = ['E282','!']

codigos_acantosis = ['L83X','!']

In [6]:
client_bq = bigquery.Client(project= proyecto)

## Funciones

In [7]:
def create_table_bq(client,query,project_id,dataset_id,table_name,replace=False,**kwargs):

    
    """
    
    This function creates (or replaces or aggregates information) a table in BigQuery from a query and stores it in a specific dataset. 
    If the table already exists, it appends the query results to the existing table.
    
    Parameters:
    - client: BigQuery API client object.
    - query: String representing the query to execute.
    - project_id: String representing the ID of the project where the dataset is located.
    - dataset_id: String representing the ID of the dataset where the table will be stored.
    - table_name: String representing the name of the table to be created or updated.
    - replace (optional): Boolean value indicating whether to replace the existing table before creating a new one.
    - **kwargs: Optional parameters to configure table creation, such as destination, write mode, 
      time partitioning, and clustering fields.
    
    Returns:
    - None
    """
    t_i = time()
    #creacion referencia tabla
    # table_ref = client.dataset(dataset_id).table(table_name)

    
    # Verificar si la tabla existe en el dataset
    try:
        print(project_id+"."+dataset_id+"."+table_name)
        client.get_table(project_id+"."+dataset_id+"."+table_name)
        existe = True
    except BaseException as e:
        print(e)
        existe = False
    
    
    if existe and replace: # si existe y se desea rempleazar
        
        print("Existe la tabla, se procede a remplazar")
        
        print(f'Borrando la tabla {kwargs["destination"]}')
        
        client.delete_table(kwargs["destination"], not_found_ok=True)#.result()
        
        if kwargs["partition_field"] != '':
            job_config = bigquery.QueryJobConfig(destination=kwargs["destination"],write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
                                                time_partitioning=bigquery.TimePartitioning(
            type_=bigquery.TimePartitioningType.MONTH, # Or HOUR
            field=kwargs["partition_field"]) # The name of the partitioning column)
            ,clustering_fields=kwargs["clustering_fields"])
        else:
            job_config = bigquery.QueryJobConfig(destination=kwargs["destination"],
                                                 write_disposition=None,                       
                                                 clustering_fields=kwargs["clustering_fields"])
        query_job =client.query(query, job_config=job_config)

        results=query_job.result()
        bytes_processed = query_job.total_bytes_processed
        gb_processed = bytes_processed / (1024 * 1024 * 1024)
        print("Cantidad de GB procesadas: ",gb_processed)
        print("-------------------------------------------")
        t_f = time()

        print("tiempo transcurrido creacion de tabla ",t_f-t_i)
    elif (existe == False and replace==True) or (existe == False and  replace==False): # si hay algun error en la entrada de parametros
        print("No existe la tabla, se procede a crearla")

        if kwargs["partition_field"] != '':
            job_config = bigquery.QueryJobConfig(destination=kwargs["destination"],
                                                write_disposition=None,
                                                time_partitioning=bigquery.TimePartitioning(type_=bigquery.TimePartitioningType.MONTH, # Or HOUR
                                                field=kwargs["partition_field"]), # The name of the partitioning column
                                                clustering_fields=kwargs["clustering_fields"])
        else:
            job_config = bigquery.QueryJobConfig(destination=kwargs["destination"],
                                                write_disposition=None,
                                                clustering_fields=kwargs["clustering_fields"])
            
            

        query_job =client.query(query, job_config=job_config)

        results=query_job.result()
        bytes_processed = query_job.total_bytes_processed
        gb_processed = bytes_processed / (1024 * 1024 * 1024)
        print("Cantidad de GB procesadas: ",gb_processed)
        print("-------------------------------------------")
        t_f = time()

        print("tiempo transcurrido creacion de tabla ",t_f-t_i)
    else: 
        print("Se annade datos a la tabla existente")
        job_config = bigquery.QueryJobConfig()
        
        job_config.destination = project_id+"."+dataset_id+"."+table_name
        job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
        query_job =client.query(query, job_config=job_config)
        
        results=query_job.result()
        bytes_processed = query_job.total_bytes_processed
        gb_processed = bytes_processed / (1024 * 1024 * 1024)
        print("Cantidad de GB procesadas: ",gb_processed)
        print("-------------------------------------------")
        t_f = time()
        print("tiempo transcurrido annadiendo data ",t_f-t_i)

## Tablas auxiliares

## Pacientes con fechas 

In [50]:
SQL_pacientes = """SELECT * FROM (SELECT numero_identificacion_paciente,
DATE(fecha_apertura_folio) as fecha_inicio,
DATE(DATE_FROM_UNIX_DATE((UNIX_DATE(DATE(fecha_apertura_folio)) - 1826))) as fecha_final,
DATE(DATE_FROM_UNIX_DATE((UNIX_DATE(DATE(fecha_apertura_folio)) - 182))) as fecha_final_imc,
diabetes,
ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY diabetes DESC) AS orden
FROM  (SELECT * FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY fecha_apertura_folio ASC) AS orden, FROM (SELECT
    numero_identificacion_paciente,
    fecha_apertura_folio,
    IF(dx_corta in ('Diabetes', '!'),1,0) as diabetes,
  FROM
    `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa`)
WHERE diabetes = 1) WHERE orden = 1

UNION ALL

SELECT * FROM (SELECT *, ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY fecha_apertura_folio DESC) AS orden, FROM (SELECT
    numero_identificacion_paciente,
    fecha_apertura_folio,
    IF(dx_corta in ('Diabetes', '!'),1,0) as diabetes,
  FROM
    `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa`)
WHERE diabetes = 0) WHERE orden = 1))
WHERE orden = 1

"""

### Creacion de la tabla

In [51]:
# Parametros de subida

nombre_tabla_pacientes = 'pacientes_identificados'

parametros_creacion = {"destination":proyecto+"."+dataset+"."+nombre_tabla_pacientes,
                    "description":"Tabla con los pacientes y las fechas para filtrar rangos de tiempo",
                    "partition_field":'',
                    "clustering_fields":['numero_identificacion_paciente']}

In [52]:
# Creacion de la tabla

create_table_bq(client=client_bq,query=SQL_pacientes,project_id=proyecto,dataset_id=dataset,table_name=nombre_tabla_pacientes,replace=True,**parametros_creacion)

co-keralty-costomedico.Diabetes_avicena_new.pacientes_identificados
Existe la tabla, se procede a remplazar
Borrando la tabla co-keralty-costomedico.Diabetes_avicena_new.pacientes_identificados
Cantidad de GB procesadas:  13.824482943862677
-------------------------------------------
tiempo transcurrido creacion de tabla  23.519474744796753


### tabla de examenes por usuarios

In [53]:
print(f"""SELECT
                              numero_identificacion_paciente,
                              fecha_apertura_folio,
                              IF(examen_laboratorio in {tuple(examenes_hdl)}, MIN(resultado_examen_laboratorio), 'No Aplica') as HDL,
                        FROM
                              `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa`
                        WHERE fecha_apertura_folio >= '{fecha_para_tomar_examenes}'
                              AND origen in ('EXAMENES LABORATORIO') 
                              --AND resultado_examen_laboratorio IS NOT NULL
                        GROUP BY    numero_identificacion_paciente, 
                                    examen_laboratorio,
                                    resultado_examen_laboratorio,
                                    fecha_apertura_folio""")

SELECT
                              numero_identificacion_paciente,
                              fecha_apertura_folio,
                              IF(examen_laboratorio in ('Colesterol HDL', 'Colesterol HDL (mg/dl)', 'HDL (mg/dl)'), MIN(resultado_examen_laboratorio), 'No Aplica') as HDL,
                        FROM
                              `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa`
                        WHERE fecha_apertura_folio >= '2019-10-31'
                              AND origen in ('EXAMENES LABORATORIO') 
                              --AND resultado_examen_laboratorio IS NOT NULL
                        GROUP BY    numero_identificacion_paciente, 
                                    examen_laboratorio,
                                    resultado_examen_laboratorio,
                                    fecha_apertura_folio


#### Query

In [54]:
## Consulta para crear tabla de examenes

SQL_examenes = f"""SELECT DISTINCT main.numero_identificacion_paciente, 
                hdl.HDL,
                ldl.LDL,
                trigliceridos.trigliceridos,
                albuminuria.albuminuria

                
FROM `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa` as main

LEFT JOIN (SELECT numero_identificacion_paciente,HDL, orden FROM (SELECT 
                              *,
                              ROW_NUMBER() over (PARTITION BY numero_identificacion_paciente ORDER BY fecha_apertura_folio desc) as orden
                        FROM(SELECT
                              main.numero_identificacion_paciente,
                              fecha_apertura_folio,
                              IF(examen_laboratorio in {tuple(examenes_hdl)}, MIN(resultado_examen_laboratorio), 'No Aplica') as HDL,
                        FROM
                              `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa` as main
                        LEFT JOIN (SELECT numero_identificacion_paciente,fecha_inicio,fecha_final FROM `co-keralty-costomedico.Diabetes_avicena_new.pacientes_identificados`) as pacientes
                        ON main.numero_identificacion_paciente = pacientes.numero_identificacion_paciente
                        WHERE DATE(main.fecha_apertura_folio) <= pacientes.fecha_inicio AND DATE(main.fecha_apertura_folio) >= pacientes.fecha_final
                              AND origen in ('EXAMENES LABORATORIO') 
                              --AND resultado_examen_laboratorio IS NOT NULL
                        GROUP BY    numero_identificacion_paciente, 
                                    examen_laboratorio,
                                    resultado_examen_laboratorio,
                                    fecha_apertura_folio)

                        WHERE HDL <>  'No Aplica') WHERE orden = 1 ) as hdl ON hdl.numero_identificacion_paciente = main.numero_identificacion_paciente

LEFT JOIN (SELECT numero_identificacion_paciente,LDL, orden FROM (SELECT 
                              *,
                              ROW_NUMBER() over (PARTITION BY numero_identificacion_paciente ORDER BY fecha_apertura_folio desc) as orden
                        FROM(SELECT
                              main.numero_identificacion_paciente,
                              fecha_apertura_folio,
                              IF(examen_laboratorio in {tuple(examenes_ldl)}, MAX(resultado_examen_laboratorio), 'No Aplica') as LDL,
                        FROM
                              `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa` as main
                        LEFT JOIN (SELECT numero_identificacion_paciente,fecha_inicio,fecha_final FROM `co-keralty-costomedico.Diabetes_avicena_new.pacientes_identificados`) as pacientes
                        ON main.numero_identificacion_paciente = pacientes.numero_identificacion_paciente
                        WHERE DATE(main.fecha_apertura_folio) <= pacientes.fecha_inicio AND DATE(main.fecha_apertura_folio) >= pacientes.fecha_final
                              AND origen in ('EXAMENES LABORATORIO') 
                              --AND resultado_examen_laboratorio IS NOT NULL
                        GROUP BY    numero_identificacion_paciente, 
                                    examen_laboratorio,
                                    resultado_examen_laboratorio,
                                    fecha_apertura_folio)

                        WHERE ldl <>  'No Aplica') WHERE orden = 1 ) as ldl ON ldl.numero_identificacion_paciente = main.numero_identificacion_paciente

LEFT JOIN (SELECT numero_identificacion_paciente,trigliceridos, orden FROM (SELECT 
                              *,
                              ROW_NUMBER() over (PARTITION BY numero_identificacion_paciente ORDER BY fecha_apertura_folio desc) as orden
                        FROM(SELECT
                              main.numero_identificacion_paciente,
                              fecha_apertura_folio,
                              IF(examen_laboratorio in {tuple(examenes_trigliceridos)}, MAX(resultado_examen_laboratorio), 'No Aplica') as trigliceridos,
                        FROM
                              `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa` as main
                        LEFT JOIN (SELECT numero_identificacion_paciente,fecha_inicio,fecha_final FROM `co-keralty-costomedico.Diabetes_avicena_new.pacientes_identificados`) as pacientes
                        ON main.numero_identificacion_paciente = pacientes.numero_identificacion_paciente
                        WHERE DATE(main.fecha_apertura_folio) <= pacientes.fecha_inicio AND DATE(main.fecha_apertura_folio) >= pacientes.fecha_final
                              AND origen in ('EXAMENES LABORATORIO') 
                              --AND resultado_examen_laboratorio IS NOT NULL
                        GROUP BY    numero_identificacion_paciente, 
                                    examen_laboratorio,
                                    resultado_examen_laboratorio,
                                    fecha_apertura_folio)

                        WHERE trigliceridos <>  'No Aplica') WHERE orden = 1 ) as trigliceridos ON trigliceridos.numero_identificacion_paciente = main.numero_identificacion_paciente

LEFT JOIN (SELECT numero_identificacion_paciente,albuminuria, orden FROM (SELECT 
                              *,
                              ROW_NUMBER() over (PARTITION BY numero_identificacion_paciente ORDER BY fecha_apertura_folio desc) as orden
                        FROM(SELECT
                              main.numero_identificacion_paciente,
                              fecha_apertura_folio,
                              IF(examen_laboratorio in {tuple(examenes_albuminuria)}, MAX(resultado_examen_laboratorio), 'No Aplica') as albuminuria,
                        FROM
                              `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa` as main
                        LEFT JOIN (SELECT numero_identificacion_paciente,fecha_inicio,fecha_final FROM `co-keralty-costomedico.Diabetes_avicena_new.pacientes_identificados`) as pacientes
                        ON main.numero_identificacion_paciente = pacientes.numero_identificacion_paciente
                        WHERE DATE(main.fecha_apertura_folio) <= pacientes.fecha_inicio AND DATE(main.fecha_apertura_folio) >= pacientes.fecha_final
                              AND origen in ('EXAMENES LABORATORIO') 
                              --AND resultado_examen_laboratorio IS NOT NULL
                        GROUP BY    numero_identificacion_paciente, 
                                    examen_laboratorio,
                                    resultado_examen_laboratorio,
                                    fecha_apertura_folio)

                        WHERE albuminuria <>  'No Aplica') WHERE orden = 1 ) as albuminuria ON albuminuria.numero_identificacion_paciente = main.numero_identificacion_paciente

                        WHERE hdl.HDL NOT IN ('PENDIENTE','VER ANEXO')
OR ldl.LDL NOT IN ('PENDIENTE','VER ANEXO','COMENTARIO')
OR trigliceridos.trigliceridos NOT IN ('PENDIENTE','VER ANEXO')
OR albuminuria.albuminuria NOT IN ('(FE)','*******','PENDIENTE')

"""

In [55]:
print(SQL_examenes)

SELECT DISTINCT main.numero_identificacion_paciente, 
                hdl.HDL,
                ldl.LDL,
                trigliceridos.trigliceridos,
                albuminuria.albuminuria

                
FROM `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa` as main

LEFT JOIN (SELECT numero_identificacion_paciente,HDL, orden FROM (SELECT 
                              *,
                              ROW_NUMBER() over (PARTITION BY numero_identificacion_paciente ORDER BY fecha_apertura_folio desc) as orden
                        FROM(SELECT
                              main.numero_identificacion_paciente,
                              fecha_apertura_folio,
                              IF(examen_laboratorio in ('Colesterol HDL', 'Colesterol HDL (mg/dl)', 'HDL (mg/dl)'), MIN(resultado_examen_laboratorio), 'No Aplica') as HDL,
                        FROM
                              `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa` as main
              

#### Creacion de la tabla

In [56]:
# Parametros de subida

nombre_tabla_examenes = 'examenes_por_usuarios'

parametros_creacion = {"destination":proyecto+"."+dataset+"."+nombre_tabla_examenes,
                    "description":"Tabla con los resultados de los examenes mas recientes para cada paciente",
                    "partition_field":'',
                    "clustering_fields":['numero_identificacion_paciente']}

In [57]:
# Creacion de la tabla

create_table_bq(client=client_bq,query=SQL_examenes,project_id=proyecto,dataset_id=dataset,table_name=nombre_tabla_examenes,replace=True,**parametros_creacion)

co-keralty-costomedico.Diabetes_avicena_new.examenes_por_usuarios
Existe la tabla, se procede a remplazar
Borrando la tabla co-keralty-costomedico.Diabetes_avicena_new.examenes_por_usuarios
Cantidad de GB procesadas:  23.819060529582202
-------------------------------------------
tiempo transcurrido creacion de tabla  23.753276824951172


### tabla de compañia por usuario

#### Query

In [58]:
SQL_compania = """SELECT
                        DISTINCT
                        numero_identificacion_paciente,
                        IF(conteo = 2, '2', compania) as compania
                        FROM (
                        SELECT
                            DISTINCT 
                            main.numero_identificacion_paciente,
                            second.conteo,
                            IF(main.compania_aseguradora IN ('30','31'), '0','1') AS compania
                        FROM
                            `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa` AS main
                        LEFT JOIN (
                            SELECT
                            numero_identificacion_paciente,
                            COUNT(1) AS conteo
                            FROM (
                            SELECT
                                DISTINCT 
                                numero_identificacion_paciente,
                                IF(compania_aseguradora IN ('30','31'), '0','1') AS compania
                            FROM
                                `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa`
                            WHERE
                                compania_aseguradora IS NOT NULL)
                            GROUP BY
                            numero_identificacion_paciente) AS second
                        ON
                            second.numero_identificacion_paciente = main.numero_identificacion_paciente
                        WHERE
                            main.compania_aseguradora IS NOT NULL
                        ORDER BY second.conteo, main.numero_identificacion_paciente DESC)"""

#### Creacion de la tabla

In [59]:
# Parametros de subida

nombre_tabla_compania = 'compania_x_usuarios'

parametros_creacion = {"destination":proyecto+"."+dataset+"."+nombre_tabla_compania,
                    "description":"Tabla con las compañias para cada paciente",
                    "partition_field":'',
                    "clustering_fields":['numero_identificacion_paciente']}

In [60]:
# Creacion de la tabla

create_table_bq(client=client_bq,query=SQL_compania,project_id=proyecto,dataset_id=dataset,table_name=nombre_tabla_compania,replace=True,**parametros_creacion)

co-keralty-costomedico.Diabetes_avicena_new.compania_x_usuarios
Existe la tabla, se procede a remplazar
Borrando la tabla co-keralty-costomedico.Diabetes_avicena_new.compania_x_usuarios
Cantidad de GB procesadas:  9.249711873009801
-------------------------------------------
tiempo transcurrido creacion de tabla  13.495841026306152


### Tabla de antecedentes por usuario

#### Query

In [61]:
SQL_antecedentes = f"""SELECT DISTINCT main.numero_identificacion_paciente,
                hipertension.med_hipertension,
                fam_dm.familiar_dm,
                cardio.ant_cardiovascular,
                dm_gestacional.dm_gestacional,
                diabetes.diabetes
                
FROM `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa` as main
LEFT JOIN 
(SELECT * FROM (SELECT
  *,
  ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY med_hipertension DESC) AS orden
FROM (
  SELECT
    numero_identificacion_paciente,
  IF
    (codigo_atc IN {tuple(codigos_medicamentos_hipertension)},1,0) AS med_hipertension,
  FROM
    `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa`))

WHERE orden = 1) as hipertension ON hipertension.numero_identificacion_paciente = main.numero_identificacion_paciente

LEFT JOIN 
(SELECT * FROM (SELECT
  *,
  ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY familiar_dm DESC) AS orden
FROM (
  SELECT
    numero_identificacion_paciente,
    IF(dx_corta in {tuple(familiar_con_dm)},1,0) as familiar_dm,
  FROM
    `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa`))

WHERE orden = 1) as fam_dm ON fam_dm.numero_identificacion_paciente = main.numero_identificacion_paciente

LEFT JOIN 
(SELECT * FROM (SELECT
  *,
  ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY ant_cardiovascular DESC) AS orden
FROM (
  SELECT
    numero_identificacion_paciente,
    IF(codigo_dx in {tuple(codigos_cardiovascular)},1,0) as ant_cardiovascular,
  FROM
    `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa`))

WHERE orden = 1) as cardio ON cardio.numero_identificacion_paciente = main.numero_identificacion_paciente

LEFT JOIN 
(SELECT * FROM (SELECT
  *,
  ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY dm_gestacional DESC) AS orden
FROM (
  SELECT
    numero_identificacion_paciente,
    IF(codigo_dx in {tuple(codigos_dm_gestacional)},1,0) as dm_gestacional,
  FROM
    `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa`))

WHERE orden = 1) as dm_gestacional ON dm_gestacional.numero_identificacion_paciente = main.numero_identificacion_paciente

LEFT JOIN 
(SELECT * FROM (SELECT
  *,
  ROW_NUMBER() OVER (PARTITION BY numero_identificacion_paciente ORDER BY diabetes DESC) AS orden
FROM (
  SELECT
    numero_identificacion_paciente,
    IF(dx_corta in {tuple(diabetes_diagnostico)},1,0) as diabetes,
  FROM
    `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa`))

WHERE orden = 1) as diabetes ON diabetes.numero_identificacion_paciente = main.numero_identificacion_paciente
"""

In [62]:
print(f"""SELECT
    numero_identificacion_paciente,
    IF(dx_corta in {tuple(diabetes_diagnostico)},1,0) as diabetes,
  FROM
    `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa`""")

SELECT
    numero_identificacion_paciente,
    IF(dx_corta in ('Diabetes', '!'),1,0) as diabetes,
  FROM
    `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa`


#### Creacion de la tabla

In [63]:
# Parametros de subida

nombre_tabla_antecedentes = 'antecedentes_x_usuario'

parametros_creacion = {"destination":proyecto+"."+dataset+"."+nombre_tabla_antecedentes,
                    "description":"Tabla con los antecedentes para cada paciente",
                    "partition_field":'',
                    "clustering_fields":['numero_identificacion_paciente']}

In [64]:
# Creacion de la tabla

create_table_bq(client=client_bq,query=SQL_antecedentes,project_id=proyecto,dataset_id=dataset,table_name=nombre_tabla_antecedentes,replace=True,**parametros_creacion)

co-keralty-costomedico.Diabetes_avicena_new.antecedentes_x_usuario
Existe la tabla, se procede a remplazar
Borrando la tabla co-keralty-costomedico.Diabetes_avicena_new.antecedentes_x_usuario
Cantidad de GB procesadas:  10.895262992940843
-------------------------------------------
tiempo transcurrido creacion de tabla  58.173907995224


### Tabla de perimetros abdominales

#### Query

In [65]:
SQL_perimetros = """SELECT numero_identificacion_paciente, PERIMETRO_ABDOMINAL, orden  FROM (SELECT
                            DISTINCT main.folio,
                            main.fecha_apertura_folio,
                            ROW_NUMBER() over (PARTITION BY main.numero_identificacion_paciente ORDER BY main.fecha_apertura_folio desc) as orden,
                            main.numero_identificacion_paciente,
                            main.genero_paciente,
                            perimetro.PERIMETRO_ABDOMINAL
                          FROM
                            `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa` AS main
                          INNER JOIN (
                            SELECT
                              DISTINCT FOLIO,
                              PERIMETRO_ABDOMINAL
                            FROM
                              `prj-dev-repository-col.bq_raw_zone_avicena.HIS_TB_EXAMEN_FISICO`
                            WHERE
                              PERIMETRO_ABDOMINAL IS NOT NULL) AS perimetro
                          ON
                            main.folio = perimetro.FOLIO
                          WHERE
                            main.fecha_apertura_folio >= '2022-10-01')

                          WHERE orden = 1"""

#### Creacion de la tabla

In [66]:
# Parametros de subida

nombre_tabla_perimetros = 'perimetros_x_usuarios'

parametros_creacion = {"destination":proyecto+"."+dataset+"."+nombre_tabla_perimetros,
                    "description":"Tabla con los antecedentes para cada paciente",
                    "partition_field":'',
                    "clustering_fields":['numero_identificacion_paciente']}

In [67]:
# Creacion de la tabla

create_table_bq(client=client_bq,query=SQL_perimetros,project_id=proyecto,dataset_id=dataset,table_name=nombre_tabla_perimetros,replace=True,**parametros_creacion)

co-keralty-costomedico.Diabetes_avicena_new.perimetros_x_usuarios
Existe la tabla, se procede a remplazar
Borrando la tabla co-keralty-costomedico.Diabetes_avicena_new.perimetros_x_usuarios
Cantidad de GB procesadas:  12.318160824477673
-------------------------------------------
tiempo transcurrido creacion de tabla  8.663187265396118


### tabla de actividad fisica por usuario

#### Query

In [68]:
SQL_ejercicio = """
SELECT
  *
FROM (
  SELECT
    DISTINCT pregunta.RESPUESTA_ANTECEDENTE,
    pregunta.PARAM_ITEM_RESPUESTA,
    respuesta.DESCRIPCION_ALTERNA AS hace_ejercicio,
    consulta.numero_identificacion_paciente,
    ROW_NUMBER() OVER (PARTITION BY consulta.numero_identificacion_paciente ORDER BY consulta.fecha_apertura_folio DESC) AS orden
  FROM
    `prj-dev-repository-col.bq_raw_zone_avicena.HIS_TB_RESP_ANTECEDEN_ITEM` AS pregunta
  INNER JOIN (
    SELECT
      *
    FROM
      `prj-dev-repository-col.bq_raw_zone_avicena.HIS_TB_ENC_PARAMETRO`) AS respuesta
  ON
    pregunta.PARAM_ITEM_RESPUESTA = respuesta.PARAMETRO
  INNER JOIN (
    SELECT
      RESPUESTA_ANTECEDENTE,
      FECHA_REGISTRO,
      FOLIO
    FROM
      `prj-dev-repository-col.bq_raw_zone_avicena.HIS_TB_RESP_ANTECEDENTE`) AS folio
  ON
    folio.RESPUESTA_ANTECEDENTE = pregunta.RESPUESTA_ANTECEDENTE
  LEFT JOIN (
    SELECT
      DISTINCT folio,
      fecha_apertura_folio,
      numero_identificacion_paciente
    FROM
      `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa`) AS consulta
  ON
    folio.FOLIO = consulta.folio
  WHERE
    pregunta.PREGUNTA = 1441
    AND pregunta.PARAM_ITEM_RESPUESTA IS NOT NULL )
WHERE
  orden = 1"""

In [69]:
print(SQL_ejercicio)


SELECT
  *
FROM (
  SELECT
    DISTINCT pregunta.RESPUESTA_ANTECEDENTE,
    pregunta.PARAM_ITEM_RESPUESTA,
    respuesta.DESCRIPCION_ALTERNA AS hace_ejercicio,
    consulta.numero_identificacion_paciente,
    ROW_NUMBER() OVER (PARTITION BY consulta.numero_identificacion_paciente ORDER BY consulta.fecha_apertura_folio DESC) AS orden
  FROM
    `prj-dev-repository-col.bq_raw_zone_avicena.HIS_TB_RESP_ANTECEDEN_ITEM` AS pregunta
  INNER JOIN (
    SELECT
      *
    FROM
      `prj-dev-repository-col.bq_raw_zone_avicena.HIS_TB_ENC_PARAMETRO`) AS respuesta
  ON
    pregunta.PARAM_ITEM_RESPUESTA = respuesta.PARAMETRO
  INNER JOIN (
    SELECT
      RESPUESTA_ANTECEDENTE,
      FECHA_REGISTRO,
      FOLIO
    FROM
      `prj-dev-repository-col.bq_raw_zone_avicena.HIS_TB_RESP_ANTECEDENTE`) AS folio
  ON
    folio.RESPUESTA_ANTECEDENTE = pregunta.RESPUESTA_ANTECEDENTE
  LEFT JOIN (
    SELECT
      DISTINCT folio,
      fecha_apertura_folio,
      numero_identificacion_paciente
    FROM
     

#### Creacion de la tabla

In [70]:
# Parametros de subida

nombre_tabla_ejercico = 'actividadFisica_x_usuarios'

parametros_creacion = {"destination":proyecto+"."+dataset+"."+nombre_tabla_ejercico,
                    "description":"Tabla con los antecedentes para cada paciente",
                    "partition_field":'',
                    "clustering_fields":['numero_identificacion_paciente']}

In [71]:
# Creacion de la tabla

create_table_bq(client=client_bq,query=SQL_ejercicio,project_id=proyecto,dataset_id=dataset,table_name=nombre_tabla_ejercico,replace=True,**parametros_creacion)

co-keralty-costomedico.Diabetes_avicena_new.actividadFisica_x_usuarios
Existe la tabla, se procede a remplazar
Borrando la tabla co-keralty-costomedico.Diabetes_avicena_new.actividadFisica_x_usuarios
Cantidad de GB procesadas:  32.34712887369096
-------------------------------------------
tiempo transcurrido creacion de tabla  8.28628396987915


## Tabla principal Diabetes Mellitus

### Tabla principal Diabetes

#### Query

In [76]:
SQL_diabetes = """SELECT * FROM  (SELECT 
                              DISTINCT
                              exam.tipo_identificacion_paciente,
                              exam.numero_identificacion_paciente,
                              MAX(DATE_DIFF(DATETIME(exam.fecha_cierre_folio),DATETIME(exam.fecha_nacimiento_paciente),YEAR)) AS edad,
                              IF(exam.genero_paciente = 'Femenino', 0, 1) as genero,
                              --exam.codigo_ciudad_sucursal,
                              exam.nivel_academico_paciente,
                              exam.raza_paciente,
                              peso.peso,
                              peso.talla,
                              peso.imc,
                              labs.HDL as HDL,
                              labs.LDL as LDL,
                              labs.trigliceridos as trigliceridos,
                              labs.albuminuria as albuminuria,
                              antecedentes.med_hipertension,
                              antecedentes.familiar_dm,
                              antecedentes.ant_cardiovascular,
                              antecedentes.dm_gestacional,
                              perimetros.PERIMETRO_ABDOMINAL,
                              ejercicio.hace_ejercicio,
                              antecedentes.diabetes
                        FROM `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa` as exam 

                    INNER JOIN (SELECT * FROM (SELECT  main.numero_identificacion_paciente,
                                        peso,
                                        talla,
                                        peso/(talla*talla) AS imc,
                                        ROW_NUMBER() over (PARTITION BY main.numero_identificacion_paciente ORDER BY fecha_apertura_folio desc) as orden
                              FROM `prj-dev-repository-col.bq_finalzone_avicena.consulta_externa` as main
                        LEFT JOIN (SELECT numero_identificacion_paciente,fecha_inicio,fecha_final_imc FROM `co-keralty-costomedico.Diabetes_avicena_new.pacientes_identificados`) as pacientes
                        ON main.numero_identificacion_paciente = pacientes.numero_identificacion_paciente
                        WHERE DATE(main.fecha_apertura_folio) <= pacientes.fecha_inicio AND DATE(main.fecha_apertura_folio) >= pacientes.fecha_final_imc
                              AND peso IS NOT NULL AND talla IS NOT NULL) WHERE orden = 1) as peso

                    ON peso.numero_identificacion_paciente = exam.numero_identificacion_paciente

                    INNER JOIN `co-keralty-costomedico.Diabetes_avicena_new.examenes_por_usuarios` as labs ON labs.numero_identificacion_paciente = exam.numero_identificacion_paciente
                    INNER JOIN `co-keralty-costomedico.Diabetes_avicena_new.perimetros_x_usuarios` as perimetros ON perimetros.numero_identificacion_paciente = exam.numero_identificacion_paciente
                    INNER JOIN `co-keralty-costomedico.Diabetes_avicena_new.actividadFisica_x_usuarios` as ejercicio ON ejercicio.numero_identificacion_paciente = exam.numero_identificacion_paciente


                  INNER JOIN `co-keralty-costomedico.Diabetes_avicena_new.antecedentes_x_usuario` as antecedentes ON antecedentes.numero_identificacion_paciente = exam.numero_identificacion_paciente

                    WHERE exam.origen = 'EXAMENES' AND 
                          exam.tipo_identificacion_paciente != 'Tarjeta de identidad' AND
                          peso.peso IS NOT NULL  
                          AND peso.talla IS NOT NULL 
                          AND exam.numero_identificacion_paciente IS NOT NULL 
                          AND exam.genero_paciente IS NOT NULL
                          AND exam.fecha_apertura_folio BETWEEN '2022-01-01' AND '2024-12-31'

                    GROUP BY 1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
                          )

                    WHERE imc > 4
                          AND HDL NOT IN ('PENDIENTE','VER ANEXO')
                          AND trigliceridos NOT IN ('PENDIENTE','VER ANEXO')
                          AND LDL NOT IN ('PENDIENTE','VER ANEXO','COMENTARIO')"""

In [73]:
print(SQL_diabetes)

SELECT * FROM  (SELECT 
                              DISTINCT
                              exam.tipo_identificacion_paciente,
                              exam.numero_identificacion_paciente,
                              MAX(DATE_DIFF(DATETIME(exam.fecha_cierre_folio),DATETIME(exam.fecha_nacimiento_paciente),YEAR)) AS edad,
                              IF(exam.genero_paciente = 'Femenino', 0, 1) as genero,
                              --exam.codigo_ciudad_sucursal,
                              exam.nivel_academico_paciente,
                              exam.raza_paciente,
                              peso.peso,
                              peso.talla,
                              peso.imc,
                              labs.HDL as HDL,
                              labs.LDL as LDL,
                              labs.trigliceridos as trigliceridos,
                              labs.albuminuria as albuminuria,
                              antecedentes.med_hipertension,
     

#### Creacion de la tabla

In [77]:
# Parametros de subida

nombre_tabla_diabetes = 'diabetes'

parametros_creacion = {"destination":proyecto+"."+dataset+"."+nombre_tabla_diabetes,
                    "description":"Tabla con los datos consolidados para el entrenamiento del modelo de diabetes",
                    "partition_field":'',
                    "clustering_fields":['numero_identificacion_paciente']}

In [78]:
# Creacion de la tabla

create_table_bq(client=client_bq,query=SQL_diabetes,project_id=proyecto,dataset_id=dataset,table_name=nombre_tabla_diabetes,replace=True,**parametros_creacion)

co-keralty-costomedico.Diabetes_avicena_new.diabetes
404 GET https://bigquery.googleapis.com/bigquery/v2/projects/co-keralty-costomedico/datasets/Diabetes_avicena_new/tables/diabetes?prettyPrint=false: Not found: Table co-keralty-costomedico:Diabetes_avicena_new.diabetes
No existe la tabla, se procede a crearla
Cantidad de GB procesadas:  31.712304127402604
-------------------------------------------
tiempo transcurrido creacion de tabla  14.139821529388428
